# HW03

### 1. Реализуем алгоритм Symspell

In [1]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter, defaultdict

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


def preprocess(text):
    sents = sentenize(text)
    return [normalize(sent.text) for sent in sents]

def ngrammer(tokens, n):
    ngrams = []
    tokens = [token for token in tokens]
    for i in range(0,len(tokens)-n+1):
        ngrams.append(tuple(tokens[i:i+n]))
    return ngrams

In [2]:
bad = open('data/sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('data/correct_sents.txt', encoding='utf8').read().splitlines()

In [3]:
# Посмотрим на пары предложений
print(bad[4])
print(true[4])

в массе своей они конечно все оччччень милые )
в массе своей они конечно все очень милые


In [4]:
# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуация на границах слов
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [5]:
pprint(align_words(true[4], bad[4]))

[('в', 'в'),
 ('массе', 'массе'),
 ('своей', 'своей'),
 ('они', 'они'),
 ('конечно', 'конечно'),
 ('все', 'все'),
 ('очень', 'оччччень'),
 ('милые', 'милые')]


Вытащим только неправильные варианты и заодно посчитаем процент ошибок.

In [6]:
mistakes = []
total = 0
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if pair[0] != pair[1]:
            mistakes.append(pair)
        total += 1

In [7]:
print('Доля ошибок - ', len(mistakes)/total )

Доля ошибок -  0.13016983016983016


Обернем в Counter, чтобы сразу увидеть частотные ошибки.

In [8]:
Counter(mistakes).most_common(10)

[(('сегодня', 'седня'), 24),
 (('вообще', 'вобще'), 18),
 (('вообще', 'ваще'), 17),
 (('естественно', 'естесственно'), 17),
 (('хочется', 'хочеться'), 16),
 (('кстати', 'кстате'), 16),
 (('очень', 'ооочень'), 14),
 (('как-то', 'както'), 9),
 (('очень', 'оооочень'), 9),
 (('это', 'ето'), 9)]

In [9]:
corpus = open('data/wiki_data.txt', encoding='utf8').read()

In [10]:
vocab = set(re.findall('\w+', corpus.lower()))

In [11]:
print(len(vocab))

368802


In [12]:
def predict_mistaken(word, vocab):

    if word in vocab:
        return 0
    else:
        return 1

In [13]:
y_true = []
y_pred = []

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if pair[0] == pair[1]:
            y_true.append(0)
        else:
            y_true.append(1)
        
        y_pred.append(predict_mistaken(pair[1], vocab))

In [14]:
WORDS = Counter(re.findall('\w+', corpus.lower()))

In [15]:
# фунцкия расчета вероятности слова
N = sum(WORDS.values())
def P(word, N=N): 
    "Вычисляем вероятность слова"
    return WORDS[word] / N

In [16]:
def edit(word):
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    return set(deletes)

In [17]:
def create_dictionary(dictionary):
    vocab = defaultdict(list)

    for word in dictionary:
        deletions = edit(word)
        for i in deletions:
            vocab[i].append(word)
    return vocab

In [18]:
def find_cand(word, dictionary):
    candidates = [word]
    word_reduced = edit(word)
    candidates.extend(dictionary[word])
    for i in word_reduced:
        candidates.extend(dictionary[i])
    return candidates

def spellcheck(word, dictionary): 
    corr_words = find_cand(word, dictionary)
    best_word = max(corr_words, key=P)
    return best_word

In [19]:
new_dict = create_dictionary(WORDS)

In [20]:
find_cand('лесница', new_dict)

['лесница', 'лестница', 'лестниц', 'легница']

In [21]:
spellcheck('лесница', new_dict)

'лестница'

In [22]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], WORDS):
            predicted = cashed.get(pair[1], spellcheck(pair[1], new_dict))
        else:
            predicted = pair[1]
        cashed[pair[1]] = predicted      
        if predicted == pair[0]:
            correct += 1
        total += 1

        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1
                
    if not i % 100:
        print(i)

0
100
200
300
400
500
600
700
800
900


In [23]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8670329670329671
0.3054489639293937
0.04892615137245894


Результат не очень хороший (всего 30 процентов ошибок исправлено)

### 2. Добавим триграммную модель

In [24]:
corpus = open('data/wiki_data.txt', encoding='utf8').read()

In [25]:
wiki_data = [['<start>', '<start>'] + sent + ['<end>'] for sent in preprocess(corpus)]

unigrams = Counter()
bigrams = Counter()
trigrams = Counter()

for sentence in wiki_data:
    unigrams.update(sentence)
    bigrams.update(ngrammer(sentence, 2))
    trigrams.update(ngrammer(sentence, 3))

In [26]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

mistakes = []

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    word_pairs = [('<start>', '<start>')] + word_pairs
    for ind, pair in enumerate(word_pairs):
        pred = None       
        if not predict_mistaken(pair[1], WORDS):
            pred = pair[1]    
        else:
            if word_pairs[ind-2][1]:
                previous_word = word_pairs[ind-2][1] + ' ' + word_pairs[ind-1][1]    
                if previous_word not in bigrams:
                    pred = spellcheck(pair[1], new_dict)
                else:
                    lm_predicted = []
                    predicted = find_cand(pair[1], new_dict)
                    for word in predicted:
                        trigram = ' '.join([previous_word, word])
                        lm_predicted.append((word, ((trigrams[trigram]/bigrams[previous_word]))))               
                    if lm_predicted:
                        pred = sorted(lm_predicted, key=lambda x: -x[1])[0][0]
        if pred is None:
            pred = pair[1] 
        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1
                
    if not i % 100:
        print(i)

0
100
200
300
400
500
600
700
800
900


In [27]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8781804869119532
0.3054489639293937
0.04426893900031175


Результат не сильно улучшился